In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pickle
from __future__ import print_function, division
from torch.optim import lr_scheduler
from skimage import io, transform
import pandas as pd

batch_size =16 
#PATH  = "/content/model"
  
print(os.getcwd())


/content


학습데이터 경로

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
class wineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로
            root_dir (string): 모든 이미지가 존재하는 디렉토리 경로
            transform (callable, optional): 샘플에 적용될 Optional transform
        """
        self.wine_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.wine_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

      
        wineFeatures = self.wine_frame.iloc[idx, :-1]
        out = np.zeros((16, 1))
        for index, point in enumerate(wineFeatures):
          out[index] = point
        dataSet = np.dot(out,out.T)
        #끝 4행에 원본데이터 입력
        for i in range(12,16):
          for index, point in enumerate(wineFeatures):
            dataSet[i][(index+i*2)%12] = point
        #끝 4열에 원본데이터 입력
        for i in range(12,16):
          for index, point in enumerate(wineFeatures):
            dataSet[(index+i*2)%12][i] = point
        #대각선 끝에 원본데이터 입력 -> 보류
        
        dataSet = np.array([dataSet])
        wineFeatureSet = dataSet.astype('double')
        
        wineQulitySet = self.wine_frame.iloc[idx, -1:]
        sample = {'wineFeatureSet': wineFeatureSet, 'wineQulitySet': wineQulitySet}
        if self.transform:
            sample = self.transform(sample)

        return sample['wineFeatureSet'], int(sample['wineQulitySet'])


class ToTensor(object):
    """numpy array를 tensor(torch)로 변환 시켜줍니다."""

    def __call__(self, sample):
        wineFeatureSet, wineQulitySet = sample['wineFeatureSet'], sample['wineQulitySet']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        result = torch.from_numpy(wineFeatureSet)
        result = result.float()
        return {'wineFeatureSet': result,
                'wineQulitySet': wineQulitySet}

In [5]:
wine_dataset = wineDataset(csv_file='/content/gdrive/My Drive/wineQualityTrainNormalization.csv',
                                    root_dir='/content/gdrive/My Drive//',
                                    transform=transforms.Compose([ ToTensor(),
                                                                 # transforms.Normalize(mean=(0.5), std=(0.5)),
                                    
                                    ])
                                    )

wine_testSet =wineDataset(csv_file='/content/gdrive/My Drive/wineQualityTestNormalization.csv',
                                    root_dir='/content/gdrive/My Drive//',
                                    transform=transforms.Compose([ ToTensor()])
                                    )
print (wine_dataset)

image, label = wine_dataset.__getitem__(5)

print(image, label)

image, label = cifar_train.__getitem__(5)

print(image,label)


tensor([[[ 1.0000,  0.0413, -0.4304, -0.7229, -0.9601, -0.7575, -0.9375,
          -0.9447, -0.5901, -0.2248, -0.5169, -0.6232,  1.0000, -0.5169,
          -0.5901, -0.9375],
         [ 0.0413,  0.0017, -0.0178, -0.0299, -0.0397, -0.0313, -0.0387,
          -0.0390, -0.0244, -0.0093, -0.0214, -0.0258,  0.0413, -0.6232,
          -0.2248, -0.9447],
         [-0.4304, -0.0178,  0.1852,  0.3111,  0.4132,  0.3260,  0.4035,
           0.4066,  0.2540,  0.0968,  0.2224,  0.2682, -0.4304,  1.0000,
          -0.5169, -0.5901],
         [-0.7229, -0.0299,  0.3111,  0.5226,  0.6941,  0.5476,  0.6777,
           0.6829,  0.4266,  0.1625,  0.3736,  0.4505, -0.7229,  0.0413,
          -0.6232, -0.2248],
         [-0.9601, -0.0397,  0.4132,  0.6941,  0.9218,  0.7273,  0.9001,
           0.9070,  0.5666,  0.2158,  0.4962,  0.5983, -0.9601, -0.4304,
           1.0000, -0.5169],
         [-0.7575, -0.0313,  0.3260,  0.5476,  0.7273,  0.5738,  0.7101,
           0.7156,  0.4470,  0.1703,  0.3915,  0.472

NameError: ignored

CNN 모델에 csv 데이터를 넣기 위한 트라이

```
# 코드로 형식 지정됨
```



*데이터* 로드, 정규화

In [6]:
cifar_train= dset.CIFAR10("CIFAR10/", train=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                            ]),
                          target_transform= None, download=True)
print (cifar_train)
print (len(cifar_train[0][0][0][0]))
print ( len(wine_dataset[0][0][0][0]))
print (len(cifar_train[0][0][0]))
print (len( wine_dataset[0][0][0]))
print (len(cifar_train[0][0]))
print ( len(wine_dataset[0][0]))
print (len(cifar_train[0]))
print ( len(wine_dataset[0]))

cifar_test= dset.CIFAR10("CIFAR10/", train=False, 
                            transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                            ]),target_transform= None, download=True)

Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10/
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: CIFAR10/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
32
16
32
16
3
1
2
2
Files already downloaded and verified


 대략적인 데이터 형태

In [0]:
def ComputeAccr(dloader, imodel):
    correct = 0 
    total= 0 
    
    for j, [imgs, labels] in enumerate(dloader):
        img  = Variable(imgs, volatile = True)
        label = Variable(labels)
        
        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)
        #print(label,output_index)
        total += label.size(0)
        correct += (output_index == label).sum().float()
        
    print("Accuracy of Test Data: {}".format(100*correct/total))
      

def TotalComputeAccr(dloader, imodel):
    classes = ('1', '2', '3', '4',
           '5', '6', '7','8','9','10',)
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    
    correct = 0 
    total= 0 
    
    for j, [imgs, labels] in enumerate(dloader):
        img  = Variable(imgs, volatile = True)
        label = Variable(labels)
        
        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)
        c = (output_index == labels).squeeze()
        for i in range(4):
            print(label[i])
            labelF = label[i]
            class_correct[labelF] += c[i].item()
            class_total[labelF] += 1



        total += label.size(0)
        correct += (output_index == label).sum().float()
        
    #print("Accuracy of Test Data: {}".format(100*correct/total))
    for i in range(10):
      print(#'Accuracy of %5s : %2d %%' % (
          classes[i], 100 * class_correct[i] , class_total[i])
      #)

데이터 로드 함수

In [0]:
train_loader = torch.utils.data.DataLoader(list(wine_dataset)[:], batch_size= batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(wine_testSet, batch_size= batch_size, shuffle=True, num_workers=2, drop_last=True)

모델 선언

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,3, padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(16),
            nn.Conv2d(16,32,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer=nn.Sequential(
            nn.Linear(64*4*4, 32),
            nn.ELU(),
            nn.Dropout2d(0.5),
            nn.BatchNorm1d(32),
            nn.Linear(32,8)
        )
        
        
    def forward(self, x):
        out= self.layer(x)
        out= out.view(batch_size,-1)
        out= self.fc_layer(out)
        return out
    
model= CNN()

loss, optimizer

In [0]:
loss_func=nn.CrossEntropyLoss()
#optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

# 불러오기

In [28]:
num_epoch_f=150
#num_epoch_m=100
#num_epoch_l=100

learning_rate = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)




epoch = 0
checkpoint = torch.load("/content/gdrive/My Drive/wineQulityNewDataModel2_1.pkl")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
TotalComputeAccr(test_loader, model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


tensor(3)
tensor(4)
tensor(5)
tensor(4)
tensor(3)
tensor(5)
tensor(4)
tensor(3)
tensor(3)
tensor(3)
tensor(4)
tensor(4)
tensor(3)
tensor(3)
tensor(7)
tensor(4)
tensor(5)
tensor(4)
tensor(3)
tensor(5)
tensor(4)
tensor(5)
tensor(3)
tensor(4)
tensor(3)
tensor(4)
tensor(6)
tensor(4)
tensor(2)
tensor(4)
tensor(5)
tensor(3)
tensor(3)
tensor(5)
tensor(4)
tensor(3)
tensor(3)
tensor(4)
tensor(4)
tensor(5)
tensor(3)
tensor(5)
tensor(4)
tensor(3)
tensor(4)
tensor(3)
tensor(5)
tensor(3)
tensor(4)
tensor(6)
tensor(2)
tensor(3)
tensor(3)
tensor(4)
tensor(3)
tensor(4)
tensor(3)
tensor(4)
tensor(4)
tensor(3)
tensor(4)
tensor(3)
tensor(4)
tensor(3)
tensor(4)
tensor(3)
tensor(3)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(4)
tensor(3)
tensor(3)
tensor(3)
tensor(4)
tensor(3)
tensor(3)
tensor(4)
tensor(5)
tensor(3)
tensor(4)
tensor(4)
tensor(3)
tensor(3)
tensor(3)
tensor(4)
tensor(5)
tensor(4)
tensor(3)
tensor(3)
tensor(3)
tensor(4)
tensor(4)
tensor(3)
tensor(3)
tensor(4)
tensor(4)


학습

In [0]:

for i in range(epoch+1 , num_epoch_f):
    print ("%d epoch :" %i) 
    model.train()
    for j,[image, label] in enumerate(train_loader):
        x= Variable(image)
        y_= Variable(label)
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_func(output, y_)
        loss.backward()
        optimizer.step()
        if j%1000 == 0:
            print(j,loss)
            # Save the model
            torch.save({
                'epoch': i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
          
            }, '/content/gdrive/My Drive/wineQulityNewDataModel2_1.pkl')

    model.eval()
    ComputeAccr(test_loader, model)

1 epoch :
0 tensor(2.0280, grad_fn=<NllLossBackward>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 49.69135665893555
2 epoch :
0 tensor(1.9939, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 50.30864334106445
3 epoch :
0 tensor(1.2045, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 50.540122985839844
4 epoch :
0 tensor(1.2286, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 49.69135665893555
5 epoch :
0 tensor(1.1995, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 49.53703689575195
6 epoch :
0 tensor(1.5192, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 48.61111068725586
7 epoch :
0 tensor(1.1080, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 49.76852035522461
8 epoch :
0 tensor(1.5078, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 50.0771598815918
9 epoch :
0 tensor(1.3008, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 50.23147964477539
10 epoch :
0 tensor(1.2454, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 50.30864334106445
11 epoch :
0 tensor(1.0957, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 50.61728286743164
12 epoc

In [0]:
중간층 64 정확도 56

In [0]:
중간층 32 정확도 55